In [1]:
import requests
import plotly.express as px

def get_food_data():
    url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        'action': 'process',
        'json': 1,
        'fields': 'product_name,brands,nutriments',
        'sort_by': 'unique_scans_n',
        'page_size': 10
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data['products']

In [4]:
def extract_food_data(data):
    product_names = [product['product_name'] for product in data]
    brands = [product.get('brands', 'Unknown') for product in data]
    calories = [product['nutriments'].get('energy-kcal_100g', 0) for product in data]
    return product_names, brands, calories

def visualize_food_data(product_names, brands, calories):
    # Bar Chart - Calories vs. Brands
    # Menampilkan kalori vs. merek, diurutkan berdasarkan kalori dari terbesar hingga terkecil
    bar_fig = px.bar(x=brands, y=calories, title='Calories vs. Brands',
                     category_orders={'x': sorted(brands, key=lambda x: -calories[brands.index(x)])})
    bar_fig.update_layout(xaxis_title='Brands', yaxis_title='Calories')
    bar_fig.update_traces(
        hovertemplate="<br>".join([
            "Brand: %{x}",
            "Calories: %{y}"
        ])
    )
    bar_fig.show()

    # Scatter Plot - Calories vs. Brands
    # Menampilkan perbandingan kalori vs. merek
    scatter_fig = px.scatter(x=calories, y=brands, title='Calories vs. Brands (Scatter Plot)')
    scatter_fig.update_layout(xaxis_title='Calories', yaxis_title='Brands')
    scatter_fig.update_traces(
        hovertemplate="<br>".join([
            "Brand: %{y}",
            "Calories: %{x}"
        ])
    )
    scatter_fig.show()

    # Pie Chart - Distribution of Calories
    # Menampilkan distribusi kalori dari produk tertentu
    pie_fig = px.pie(values=calories, names=product_names, 
                     title='Calories Distribution',
                     category_orders={'x': sorted(product_names, key=lambda x: -calories[product_names.index(x)])})
    pie_fig.update_traces(customdata=brands, hovertemplate='Product: %{label}<br>Brand: %{customdata}<br>Calories: %{value}')
    pie_fig.show()

    # Treemap - Top Products by Calories
    # Menampilkan produk teratas berdasarkan kalori
    treemap_fig = px.treemap(names=brands, parents=[''] * len(brands), values=calories,
                             title='Top Products by Calories',
                             hover_data={'labels': product_names, 'values': calories, 'brands': brands})
    treemap_fig.update_traces(hovertemplate='Product: %{label}<br>Brand: %{customdata[0]}<br>Calories: %{value}')
    treemap_fig.show()

if __name__ == "__main__":
    food_data = get_food_data()
    product_names, brands, calories = extract_food_data(food_data)
    visualize_food_data(product_names, brands, calories)
